## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-23-08-46-34 +0000,nypost,Father and son’s metal detecting trip uncovers...,https://nypost.com/2025/11/23/world-news/czech...
1,2025-11-23-08-40-00 +0000,bbc,Rail fares to be frozen in England next year,https://www.bbc.com/news/articles/cwygx71g3n7o...
2,2025-11-23-08-32-51 +0000,bbc,Escalating war of words between Ethiopia and E...,https://www.bbc.com/news/articles/cvgwg042vr4o...
3,2025-11-23-08-00-17 +0000,nyt,Hong Kong’s Global Style: How Locals and Touri...,https://www.nytimes.com/2025/11/23/style/hong-...
4,2025-11-23-07-42-57 +0000,bbc,JFK's granddaughter shares terminal cancer dia...,https://www.bbc.com/news/articles/cpvdyg348pko...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2362/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
97,trump,31
50,ukraine,14
52,plan,12
124,greene,11
122,marjorie,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
136,2025-11-22-15-51-00 +0000,wsj,"Republican Marjorie Taylor Greene, who fought ...",https://www.wsj.com/politics/policy/marjorie-t...,93
171,2025-11-22-12-15-08 +0000,nypost,Trump slams ‘traitor’ Rep. Marjorie Taylor Gre...,https://nypost.com/2025/11/22/us-news/trump-ce...,93
158,2025-11-22-13-26-13 +0000,cbc,Canada joins world leaders saying Trump’s plan...,https://www.cbc.ca/news/politics/carney-g20-su...,81
155,2025-11-22-13-47-58 +0000,nyt,Marjorie Taylor Greene Says She Will Resign in...,https://www.nytimes.com/2025/11/21/us/politics...,77
15,2025-11-23-04-36-28 +0000,nyt,Georgia Voters Stand by Marjorie Taylor Greene...,https://www.nytimes.com/2025/11/22/us/georgia-...,76


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
136,93,2025-11-22-15-51-00 +0000,wsj,"Republican Marjorie Taylor Greene, who fought ...",https://www.wsj.com/politics/policy/marjorie-t...
23,59,2025-11-23-03-38-58 +0000,nypost,Marco Rubio told US senators that Ukraine peac...,https://nypost.com/2025/11/22/world-news/marco...
160,43,2025-11-22-13-18-45 +0000,startribune,She wants to play varsity basketball at a neig...,https://www.startribune.com/she-wants-to-play-...
141,39,2025-11-22-15-03-50 +0000,nypost,More Kennedy tragedy strikes as Tatiana Schlos...,https://nypost.com/2025/11/22/us-news/tatiana-...
133,38,2025-11-22-16-00-27 +0000,nypost,‘Ready to fight’: Hamas reportedly says it’s p...,https://nypost.com/2025/11/22/world-news/hamas...
104,34,2025-11-22-19-44-00 +0000,wsj,President Trump said Friday he was immediately...,https://www.wsj.com/politics/policy/trump-says...
60,29,2025-11-22-23-58-55 +0000,nypost,Bodycam footage shows Nevada stepdad scream ‘m...,https://nypost.com/2025/11/22/us-news/bodycam-...
139,29,2025-11-22-15-12-00 +0000,wsj,Owner of U.K.’s Daily Mail Nears Deal to Buy R...,https://www.wsj.com/business/media/owner-of-u-...
143,29,2025-11-22-15-00-57 +0000,nyt,Jair Bolsonaro Arrested in Brazil Amid Fears H...,https://www.nytimes.com/2025/11/22/world/ameri...
95,28,2025-11-22-20-39-00 +0000,wsj,Boston Fed President says she sees no strong n...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
